In [1]:
import sys

assert sys.version_info >= (3, 7)

In [2]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [3]:
from pathlib import Path

In [4]:
# define the default font sizes to make the figures prettier:

In [8]:
tf.random.set_seed(42)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(tf.keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(tf.keras.layers.BatchNormalization()) # Batch normalization requires adding an activation layer after
    model.add(tf.keras.layers.Activation("swish"))

model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [9]:
dataset = tf.keras.datasets.cifar10.load_data()

In [10]:
optimizer = tf.keras.optimizers.legacy.Nadam(learning_rate=5e-4) #Legacy cause made on a m2
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [11]:
(X_train_full, y_train_full), (X_test, y_test) = dataset

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [12]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20,
                                                     restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5",
                                                         save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = Path() / "my_cifar10_logs" / f"run_{run_index:03d}"
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [13]:
%load_ext tensorboard
%tensorboard --logdir=./my_cifar10_logs

Reusing TensorBoard on port 6006 (pid 10642), started 0:12:09 ago. (Use '!kill 10642' to kill it.)

In [14]:
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 13s 7ms/step - loss: 2.0332 - accuracy: 0.2481 - val_loss: 2.2062 - val_accuracy: 0.2396
Epoch 2/100
  17/1407 [..............................] - ETA: 9s - loss: 1.8018 - accuracy: 0.3621 

/opt/anaconda3/envs/homl3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1407/1407 [==============================] - 10s 7ms/step - loss: 1.7756 - accuracy: 0.3606 - val_loss: 1.8914 - val_accuracy: 0.3276
Epoch 3/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.6755 - accuracy: 0.3988 - val_loss: 2.0865 - val_accuracy: 0.2620
Epoch 4/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.6208 - accuracy: 0.4214 - val_loss: 1.6423 - val_accuracy: 0.4186
Epoch 5/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.5524 - accuracy: 0.4508 - val_loss: 1.7294 - val_accuracy: 0.3988
Epoch 6/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.5069 - accuracy: 0.4644 - val_loss: 1.5678 - val_accuracy: 0.4310
Epoch 7/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.4611 - accuracy: 0.4801 - val_loss: 1.6743 - val_accuracy: 0.4212
Epoch 8/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.4215 - accuracy: 0.4930 - val_loss: 1.5522 - val_accurac

In [15]:
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 2ms/step - loss: 1.4352 - accuracy: 0.5144


[1.4351547956466675, 0.5144000053405762]